In [ ]:
# always check your python version
!python --version

In [ ]:
# AWS SDK for Python, -q flag is for quiet
%pip install -q boto3

In [ ]:
# always check your boto3 version
%pip show boto3

In [ ]:
import boto3
client = boto3.client("bedrock-runtime")

In [ ]:
# This is a prompt optimized for Titan.
# https://d2eo22ngex1n9g.cloudfront.net/Documentation/User+Guides/Titan/Amazon+Titan+Text+Prompt+Engineering+Guidelines.pdf
prompt_titan = """
User: 
Sulfuric acid reacts with sodium chloride, and gives {{chemical1}} and {{chemical2}}:
Bot:
"""

# This is a prompt optimized for Claude 
# https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/overview
prompt_claude = """
Human: 
Sulfuric acid reacts with sodium chloride, and gives <chemical1>_____</chemical1> and <chemical2>_____</chemical2>:
Assistant:
"""

In [ ]:
# Before you can invoke the model you need to enable model access in Amazon Bedrock

import json
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/invoke_model.html
resp = client.invoke_model(
  modelId='amazon.titan-text-express-v1',
  body=json.dumps({
    "inputText": prompt
  })
)
body = json.loads(resp.get("body").read())
print(body['results'][0]['outputText'])

In [ ]:
# Notice how the body and output structure is different
# Notice how XML is much better handled by Claude Haiku 3
# https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/use-xml-tags

# In the Workshop Studio we don't have access to calude v2
#resp = client.invoke_model(
#    modelId='anthropic.claude-v2',
#    contentType='application/json',
#    accept='application/json',
#    body=json.dumps({
#        "prompt": prompt,
#        "max_tokens_to_sample": 200,
#        "temperature": 1.0
#    })
#)
#body = json.loads(resp.get("body").read())
#print(body['completion'])


resp = client.invoke_model(
    modelId='anthropic.claude-3-haiku-20240307-v1:0',
    contentType='application/json',
    accept='application/json',
    body=json.dumps({
        "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}],
        "max_tokens": 200,
        "temperature": 1.0,
        "anthropic_version": "bedrock-2023-05-31"
    })
)
body = json.loads(resp.get("body").read())
print(body['content'][0]['text'])

In [ ]:
# Lets create a helper.py and a text file loader
from helper import load_text_file
prompt = load_text_file("prompts/zero-shot.txt")
print(prompt)

In [ ]:
resp = client.invoke_model(
    modelId='anthropic.claude-3-haiku-20240307-v1:0',
    contentType='application/json',
    accept='application/json',
    body=json.dumps({
        "messages": [{"role": "user", "content": [{"type": "text", "text": prompt}]}],
        "max_tokens": 200,
        "temperature": 1.0,
        "anthropic_version": "bedrock-2023-05-31"
    })
)
body = json.loads(resp.get("body").read())
print(body['content'][0]['text'])